Here, you develop the 

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

This cell takes all of the input parameters.

In [2]:
nh = 16
refRatio = 2
c = 1.
CFL = 0.5
nt = 32
RK = 4
deriv = 'CD'
order = 2

This cell contains the relevant switch information.

In [3]:
if (RK == 1):
    TimeIntegratorFunc = ST.ForwardEuler
else:
    if (RK == 2):
        TimeIntegratorFunc = ST.MidpointMeth
    else:
        TimeIntegratorFunc = ST.RK4

if (deriv == 'U'):
    DiffFunc = ST.Upwind
else:
    DiffFunc = TT.CenterDiff#ST.CenterDiff

This cell instantiates the grid, derivative operators, and Fourier modes.

In [4]:
omega = BT.Grid(nh)
finehalf = list(np.arange(int(nh / refRatio)))# + int(nh / refRatio))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed
nh_min = omega.nh_min
nh_max = omega.nh_max
t, nt = ST.CalcTime(omega, CFL, c, nt = nt)
derivMat = OT.SpaceDeriv(omega, order, deriv)
spatOp = -c * derivMat
strings = omega.strings
timePropOp = LA2.expm(t * spatOp)
omegaF = BT.Grid(nh_max)
wavesF = WT.MakeWaves(omegaF)

In [5]:
polyInterp = GTT.GhostCellStencil(2, -0.5)
print(polyInterp)
print(type(polyInterp))

(array([ 0.5 ,  0.75, -0.25]), 1, 2)
<class 'tuple'>


This cell runs a test on a fifth-order cell-averaged polynomial interpolation for a ghost cell at the edge of the coarse-fine interface. If it doesn't reach the appropriate order of accuracy, there is an `assert` which force quits.

In [6]:
TT.TestPoly(5, 2)

Order 0:
p(x) =
  
2
P(x) =
  
2 x
2.0 1.9999999999999973
Error = -2.6645352591003757e-15

Order 1:
p(x) =
  
2 x
P(x) =
    2
1 x
3.5 3.5000000000000044
Error = 4.440892098500626e-15

Order 2:
p(x) =
    2
2 x
P(x) =
         3
0.6667 x
6.166666666666666 6.166666666666671
Error = 5.329070518200751e-15

Order 3:
p(x) =
    3
2 x
P(x) =
      4
0.5 x
10.9375 10.93750000000001
Error = 1.0658141036401503e-14

Order 4:
p(x) =
    4
2 x
P(x) =
      5
0.4 x
19.525 19.525000000000027
Error = 2.842170943040401e-14

Order 5:
p(x) =
    5
2 x
P(x) =
         6
0.3333 x
35.072916666666664 35.072916666666714
Error = 4.973799150320701e-14

Order 6:
p(x) =
    6
2 x
P(x) =
         7
0.2857 x
63.379464285714285 59.36160714285724
Error = -4.017857142857046



In [7]:
order = 3
for i in range(10):
    x_0 = 0.5 * (i + 1)
    print(x_0)
    print(np.polynomial.polynomial.polyvander(x_0, order)[0][::-1])
    print('')

0.5
[0.125 0.25  0.5   1.   ]

1.0
[1. 1. 1. 1.]

1.5
[3.375 2.25  1.5   1.   ]

2.0
[8. 4. 2. 1.]

2.5
[15.625  6.25   2.5    1.   ]

3.0
[27.  9.  3.  1.]

3.5
[42.875 12.25   3.5    1.   ]

4.0
[64. 16.  4.  1.]

4.5
[91.125 20.25   4.5    1.   ]

5.0
[125.  25.   5.   1.]



In [8]:
A = np.arange(10)
print(A)
print(A[:-1])

[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8]
